In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datamol as dm

from rdkit import Chem
from rdkit.Chem import Draw, PandasTools, MACCSkeys, rdFingerprintGenerator

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict, cross_val_score
from tpot import TPOTRegressor

from pathlib import Path
import pickle

d:\Visual Studio 2019\Python\.rdkit\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
pd.options.display.max_columns = 50

In [3]:
data = pd.read_csv('data_clean.csv', index_col=0)
data.head()

,molecule_chembl_id,Ki,units,pKi,smiles,molecular_weight,n_hba,n_hbd,logp,passed
0,CHEMBL132806,57.0,nM,7.24,CC(C)CCn1cc2c(nc(NC(=O)Cc3ccccc3)n3nc(-c4ccco4...,429.191323,9,1,3.9613,True
1,CHEMBL336217,42.0,nM,7.38,CC(C)CCn1cc2c(nc(NC(=O)Cc3cccc4ccccc34)n3nc(-c...,479.206973,9,1,5.1145,True
2,CHEMBL134566,60.0,nM,7.22,O=C(Cc1ccccc1)Nc1nc2nn(CCCc3ccccc3)cc2c2nc(-c3...,477.191323,9,1,4.5480,True
3,CHEMBL435022,38.0,nM,7.42,CC(C)CCn1cc2c(nc(NC(=O)COc3ccccc3)n3nc(-c4ccco...,445.186238,10,1,3.7976,True
4,CHEMBL341376,423.0,nM,6.37,Cn1cc2c(nc(NC(=O)Cc3ccccc3)n3nc(-c4ccco4)nc23)n1,373.128723,9,1,2.4522,True


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2943 entries, 0 to 2942
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   molecule_chembl_id  2943 non-null   object 
 1   Ki                  2943 non-null   float64
 2   units               2943 non-null   object 
 3   pKi                 2943 non-null   float64
 4   smiles              2943 non-null   object 
 5   molecular_weight    2943 non-null   float64
 6   n_hba               2943 non-null   int64  
 7   n_hbd               2943 non-null   int64  
 8   logp                2943 non-null   float64
 9   passed              2943 non-null   bool   
dtypes: bool(1), float64(4), int64(2), object(3)
memory usage: 232.8+ KB


In [5]:
data = data[list(data.columns)[:5]]
data.head()

,molecule_chembl_id,Ki,units,pKi,smiles
0,CHEMBL132806,57.0,nM,7.24,CC(C)CCn1cc2c(nc(NC(=O)Cc3ccccc3)n3nc(-c4ccco4...
1,CHEMBL336217,42.0,nM,7.38,CC(C)CCn1cc2c(nc(NC(=O)Cc3cccc4ccccc34)n3nc(-c...
2,CHEMBL134566,60.0,nM,7.22,O=C(Cc1ccccc1)Nc1nc2nn(CCCc3ccccc3)cc2c2nc(-c3...
3,CHEMBL435022,38.0,nM,7.42,CC(C)CCn1cc2c(nc(NC(=O)COc3ccccc3)n3nc(-c4ccco...
4,CHEMBL341376,423.0,nM,6.37,Cn1cc2c(nc(NC(=O)Cc3ccccc3)n3nc(-c4ccco4)nc23)n1


In [6]:
PandasTools.AddMoleculeColumnToFrame(data, smilesCol='smiles', molCol='mols')

In [7]:
data['maccs'] = data['mols'].apply(MACCSkeys.GenMACCSKeys)

In [8]:
data.head()

,molecule_chembl_id,Ki,units,pKi,smiles,mols,maccs
0,CHEMBL132806,57.0,nM,7.24,CC(C)CCn1cc2c(nc(NC(=O)Cc3ccccc3)n3nc(-c4ccco4...,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,CHEMBL336217,42.0,nM,7.38,CC(C)CCn1cc2c(nc(NC(=O)Cc3cccc4ccccc34)n3nc(-c...,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,CHEMBL134566,60.0,nM,7.22,O=C(Cc1ccccc1)Nc1nc2nn(CCCc3ccccc3)cc2c2nc(-c3...,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,CHEMBL435022,38.0,nM,7.42,CC(C)CCn1cc2c(nc(NC(=O)COc3ccccc3)n3nc(-c4ccco...,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,CHEMBL341376,423.0,nM,6.37,Cn1cc2c(nc(NC(=O)Cc3ccccc3)n3nc(-c4ccco4)nc23)n1,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
dm.descriptors.descriptors.Descriptors.descList

[('MaxAbsEStateIndex',
  <function rdkit.Chem.EState.EState.MaxAbsEStateIndex(mol, force=1)>),
 ('MaxEStateIndex',
  <function rdkit.Chem.EState.EState.MaxEStateIndex(mol, force=1)>),
 ('MinAbsEStateIndex',
  <function rdkit.Chem.EState.EState.MinAbsEStateIndex(mol, force=1)>),
 ('MinEStateIndex',
  <function rdkit.Chem.EState.EState.MinEStateIndex(mol, force=1)>),
 ('qed',
  <function rdkit.Chem.QED.qed(mol, w=QEDproperties(MW=0.66, ALOGP=0.46, HBA=0.05, HBD=0.61, PSA=0.06, ROTB=0.65, AROM=0.48, ALERTS=0.95), qedProperties=None)>),
 ('SPS', <function rdkit.Chem.SpacialScore.SPS(mol, normalize=True)>),
 ('MolWt', <function rdkit.Chem.Descriptors.<lambda>(*x, **y)>),
 ('HeavyAtomMolWt', <function rdkit.Chem.Descriptors.HeavyAtomMolWt(x)>),
 ('ExactMolWt', <function rdkit.Chem.Descriptors.<lambda>(*x, **y)>),
 ('NumValenceElectrons',
  <function rdkit.Chem.Descriptors.NumValenceElectrons(mol)>),
 ('NumRadicalElectrons',
  <function rdkit.Chem.Descriptors.NumRadicalElectrons(mol)>),
 ('Ma

This is the list of 210 descriptors. Which descriptors are the most important?

In [10]:
len(dm.descriptors.descriptors.Descriptors.descList)

210

In [11]:
all_desc_df = dm.descriptors.batch_compute_many_descriptors(mols=data['mols'].to_list(), progress=True, batch_size='auto')

  0%|          | 0/2943 [00:00<?, ?it/s]

In [12]:
all_desc_df.head()

,mw,fsp3,n_lipinski_hba,n_lipinski_hbd,n_rings,n_hetero_atoms,n_heavy_atoms,n_rotatable_bonds,n_radical_electrons,tpsa,qed,clogp,sas,n_aliphatic_carbocycles,n_aliphatic_heterocyles,n_aliphatic_rings,n_aromatic_carbocycles,n_aromatic_heterocyles,n_aromatic_rings,n_saturated_carbocycles,n_saturated_heterocyles,n_saturated_rings
0,429.191323,0.260870,9,1,5,9,32,7,0,103.14,0.420000,3.9613,2.785385,0,0,0,1,4,5,0,0,0
1,479.206973,0.222222,9,1,6,9,36,7,0,103.14,0.341105,5.1145,2.869417,0,0,0,2,4,6,0,0,0
2,477.191323,0.148148,9,1,6,9,36,8,0,103.14,0.345039,4.5480,2.702531,0,0,0,2,4,6,0,0,0
3,445.186238,0.260870,10,1,5,10,33,8,0,112.37,0.386515,3.7976,2.792537,0,0,0,1,4,5,0,0,0
4,373.128723,0.105263,9,1,5,9,28,4,0,103.14,0.518988,2.4522,2.681130,0,0,0,1,4,5,0,0,0


In [13]:
all_desc_df['n_radical_electrons'].unique()

array([0])

In [14]:
all_desc_df.drop(labels= 'n_radical_electrons', axis='columns', inplace=True)

In [15]:
data = pd.concat([data, all_desc_df], axis=1)
data.head()

,molecule_chembl_id,Ki,units,pKi,smiles,mols,maccs,mw,fsp3,n_lipinski_hba,n_lipinski_hbd,n_rings,n_hetero_atoms,n_heavy_atoms,n_rotatable_bonds,tpsa,qed,clogp,sas,n_aliphatic_carbocycles,n_aliphatic_heterocyles,n_aliphatic_rings,n_aromatic_carbocycles,n_aromatic_heterocyles,n_aromatic_rings,n_saturated_carbocycles,n_saturated_heterocyles,n_saturated_rings
0,CHEMBL132806,57.0,nM,7.24,CC(C)CCn1cc2c(nc(NC(=O)Cc3ccccc3)n3nc(-c4ccco4...,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",429.191323,0.260870,9,1,5,9,32,7,103.14,0.420000,3.9613,2.785385,0,0,0,1,4,5,0,0,0
1,CHEMBL336217,42.0,nM,7.38,CC(C)CCn1cc2c(nc(NC(=O)Cc3cccc4ccccc34)n3nc(-c...,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",479.206973,0.222222,9,1,6,9,36,7,103.14,0.341105,5.1145,2.869417,0,0,0,2,4,6,0,0,0
2,CHEMBL134566,60.0,nM,7.22,O=C(Cc1ccccc1)Nc1nc2nn(CCCc3ccccc3)cc2c2nc(-c3...,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",477.191323,0.148148,9,1,6,9,36,8,103.14,0.345039,4.5480,2.702531,0,0,0,2,4,6,0,0,0
3,CHEMBL435022,38.0,nM,7.42,CC(C)CCn1cc2c(nc(NC(=O)COc3ccccc3)n3nc(-c4ccco...,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",445.186238,0.260870,10,1,5,10,33,8,112.37,0.386515,3.7976,2.792537,0,0,0,1,4,5,0,0,0
4,CHEMBL341376,423.0,nM,6.37,Cn1cc2c(nc(NC(=O)Cc3ccccc3)n3nc(-c4ccco4)nc23)n1,<rdkit.Chem.rdchem.Mol object at 0x0000018B5EA...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",373.128723,0.105263,9,1,5,9,28,4,103.14,0.518988,2.4522,2.681130,0,0,0,1,4,5,0,0,0


In [16]:
data.shape

(2943, 28)

All rows in ' 'n_radical_electrons' column are equal to 0.0, so we can drop it safely

In [17]:
all_desc_df.columns

Index(['mw', 'fsp3', 'n_lipinski_hba', 'n_lipinski_hbd', 'n_rings',
       'n_hetero_atoms', 'n_heavy_atoms', 'n_rotatable_bonds', 'tpsa', 'qed',
       'clogp', 'sas', 'n_aliphatic_carbocycles', 'n_aliphatic_heterocyles',
       'n_aliphatic_rings', 'n_aromatic_carbocycles', 'n_aromatic_heterocyles',
       'n_aromatic_rings', 'n_saturated_carbocycles',
       'n_saturated_heterocyles', 'n_saturated_rings'],
      dtype='object')

In [18]:
'n_radical_electrons' in data.columns

False

In [19]:
X_all = np.stack(data['maccs'])
X_all.shape

(2943, 167)

In [20]:
col_y = ['pKi'] + list(all_desc_df.columns[1:])
col_y

['pKi',
 'fsp3',
 'n_lipinski_hba',
 'n_lipinski_hbd',
 'n_rings',
 'n_hetero_atoms',
 'n_heavy_atoms',
 'n_rotatable_bonds',
 'tpsa',
 'qed',
 'clogp',
 'sas',
 'n_aliphatic_carbocycles',
 'n_aliphatic_heterocyles',
 'n_aliphatic_rings',
 'n_aromatic_carbocycles',
 'n_aromatic_heterocyles',
 'n_aromatic_rings',
 'n_saturated_carbocycles',
 'n_saturated_heterocyles',
 'n_saturated_rings']

In [21]:
y_all = data[col_y]
y_all.shape

(2943, 21)

In [22]:
X_1, X_test, y_1, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

In [23]:
X_train, X_val, y_train, y_val = train_test_split(X_1, y_1, test_size=0.2, random_state=42)

In [24]:
X_train.shape, X_val.shape, X_test.shape

((1883, 167), (471, 167), (589, 167))

In [25]:
y_train.shape, y_val.shape, y_test.shape

((1883, 21), (471, 21), (589, 21))

In [26]:
rf_reg = RandomForestRegressor(n_estimators=100, criterion='squared_error', n_jobs=-1, random_state=42)

In [27]:
rf_cross = []
for i in range(y_train.shape[1]):
    score = - cross_val_score(rf_reg, X_train, y_train.iloc[:, i], cv=5, n_jobs=-1, scoring='neg_root_mean_squared_error')
    rf_cross.append(score.mean())

In [28]:
rf_cross

[np.float64(0.739325574048611),
 np.float64(0.04403989752992295),
 np.float64(0.5529716940359378),
 np.float64(0.20027233561372465),
 np.float64(0.3733946817711795),
 np.float64(0.6595744210214618),
 np.float64(2.21577491893581),
 np.float64(0.7287246969827195),
 np.float64(7.191903227596302),
 np.float64(0.06738606957080148),
 np.float64(0.6292274173296046),
 np.float64(0.1986290203400007),
 np.float64(0.17760858547665243),
 np.float64(0.1705741020494557),
 np.float64(0.2466878665920222),
 np.float64(0.3716329611714874),
 np.float64(0.32814529386286273),
 np.float64(0.37465512634675985),
 np.float64(0.1517909474493001),
 np.float64(0.13688993242264055),
 np.float64(0.20336124948047582)]

In [29]:
rf_cros_df = pd.Series(rf_cross, index=col_y, name='random_forest')
rf_cros_df

pKi                        0.739326
fsp3                       0.044040
n_lipinski_hba             0.552972
n_lipinski_hbd             0.200272
n_rings                    0.373395
n_hetero_atoms             0.659574
n_heavy_atoms              2.215775
n_rotatable_bonds          0.728725
tpsa                       7.191903
qed                        0.067386
clogp                      0.629227
sas                        0.198629
n_aliphatic_carbocycles    0.177609
n_aliphatic_heterocyles    0.170574
n_aliphatic_rings          0.246688
n_aromatic_carbocycles     0.371633
n_aromatic_heterocyles     0.328145
n_aromatic_rings           0.374655
n_saturated_carbocycles    0.151791
n_saturated_heterocyles    0.136890
n_saturated_rings          0.203361
Name: random_forest, dtype: float64

In [30]:
rf_reg.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=42)

In [50]:
rf_pred = rf_reg.predict(X_val)

In [56]:
rf_pred

array([[8.33060000e+00, 2.53988963e-01, 9.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.27740000e+00, 3.28990769e-01, 1.09500000e+01, ...,
        3.20000000e-01, 3.70000000e-01, 6.90000000e-01],
       [7.66830000e+00, 4.40247705e-02, 5.71000000e+00, ...,
        0.00000000e+00, 2.00000000e-02, 2.00000000e-02],
       ...,
       [8.66390833e+00, 1.45866397e-01, 7.05000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.80380000e+00, 1.34256226e-01, 4.44000000e+00, ...,
        0.00000000e+00, 1.00000000e-02, 1.00000000e-02],
       [5.90436333e+00, 3.57078431e-01, 5.23000000e+00, ...,
        9.80000000e-01, 1.00000000e-02, 9.90000000e-01]])

In [57]:
rf_pred.shape

(471, 21)

In [58]:
y_val.shape

(471, 21)

In [65]:
y_val.to_numpy()[:, 0]

array([ 8.73,  6.22,  8.11,  6.84,  5.22,  5.51,  6.72,  7.  ,  6.05,
        9.  ,  7.19,  5.95,  9.23,  7.  ,  6.07,  7.51,  5.38,  8.15,
        5.8 ,  7.93,  5.47,  6.46,  5.23,  7.94,  7.15,  7.77,  4.96,
        5.15,  7.05,  5.64,  7.17,  6.85,  7.3 ,  6.99,  5.57,  5.98,
        4.32,  6.48,  9.52,  7.57,  6.71,  7.01,  7.75,  7.92,  8.3 ,
        5.87,  5.11,  6.96,  8.5 ,  7.24,  9.38,  7.77,  7.08,  4.89,
        7.29,  7.96,  8.08,  6.71,  7.75,  7.3 ,  7.92,  8.89,  5.11,
        6.38,  6.96,  7.7 ,  7.94,  5.17,  6.4 ,  5.66,  5.54,  6.07,
        7.58,  6.22,  6.05,  6.88,  8.24,  6.89,  8.85,  8.68,  4.89,
        6.72,  5.5 ,  7.8 ,  5.52,  7.11,  9.08,  7.62,  7.11,  5.39,
        5.6 ,  5.19,  8.1 ,  6.76,  5.39,  5.7 ,  5.43,  7.55,  7.66,
        6.38,  7.23,  6.1 ,  8.46,  6.03,  9.15,  5.99, 10.09,  7.03,
        6.15,  9.22,  7.85,  7.77,  6.45,  6.52,  7.  ,  4.99,  7.23,
        7.58,  7.38,  7.26,  7.24,  8.26,  6.01,  7.85,  5.34,  8.68,
        7.26,  8.3 ,

In [64]:
rf_pred[:, 0]

array([8.3306    , 6.2774    , 7.6683    , 6.6881    , 6.61632   ,
       5.2134    , 6.98029   , 8.807     , 6.69020333, 8.6153    ,
       7.1007    , 6.34685333, 7.63074   , 7.562725  , 5.86645   ,
       8.06056667, 5.85516667, 7.48466667, 5.959     , 7.98995   ,
       5.46945   , 5.7908    , 5.6994    , 8.48166667, 7.43377667,
       7.26246222, 5.7527    , 6.28121667, 7.38146   , 6.52009167,
       6.21653667, 7.05770833, 6.82088333, 6.63814   , 5.95285   ,
       6.79142667, 6.58950286, 6.56561452, 7.63591667, 7.05986   ,
       7.0862    , 6.50073333, 7.41386833, 6.39360857, 7.95492333,
       5.3715    , 5.7519    , 6.16448333, 8.1744    , 7.61958333,
       8.244     , 7.6953    , 7.02633333, 6.5907    , 6.32156667,
       6.9159    , 7.5149    , 6.682     , 6.6019    , 7.5617    ,
       7.15455   , 7.70766667, 6.86358667, 6.57671667, 6.46833333,
       6.9015    , 7.66543333, 6.07103333, 7.78746583, 5.3839    ,
       5.93974167, 5.79586833, 7.2791    , 6.5756    , 6.7798 

In [80]:
procentual = []
for i in range(y_train.shape[1]):
    # print(y_train.iloc[:, i].name)
    y_true = y_val.iloc[:, i].to_numpy()
    proc = (y_true - rf_pred[:, i]) 
    print(y_train.iloc[:, i].name, proc.max())

pKi 2.3995500000000005
fsp3 0.3395646402960645
n_lipinski_hba 3.241666666666667
n_lipinski_hbd 1.85
n_rings 1.42
n_hetero_atoms 3.991666666666667
n_heavy_atoms 9.510000000000002
n_rotatable_bonds 6.140000000000001
tpsa 34.25870000000006
qed 0.29053072726328266
clogp 2.7537402000000046
sas 1.7131127772628165
n_aliphatic_carbocycles 1.3900000000000001
n_aliphatic_heterocyles 1.98
n_aliphatic_rings 2.05
n_aromatic_carbocycles 1.3156666666666665
n_aromatic_heterocyles 1.645
n_aromatic_rings 1.6099999999999999
n_saturated_carbocycles 1.4100000000000001
n_saturated_heterocyles 1.98
n_saturated_rings 1.38


In [31]:
svm_reg = SVR(kernel='rbf', C=1)

In [32]:
res_svm_cross = []

In [33]:
y_train.shape[1]

21

In [34]:
for i in range(y_train.shape[1]):
    b = y_train.iloc[:, i]
    print(b.shape, b.name)

(1883,) pKi
(1883,) fsp3
(1883,) n_lipinski_hba
(1883,) n_lipinski_hbd
(1883,) n_rings
(1883,) n_hetero_atoms
(1883,) n_heavy_atoms
(1883,) n_rotatable_bonds
(1883,) tpsa
(1883,) qed
(1883,) clogp
(1883,) sas
(1883,) n_aliphatic_carbocycles
(1883,) n_aliphatic_heterocyles
(1883,) n_aliphatic_rings
(1883,) n_aromatic_carbocycles
(1883,) n_aromatic_heterocyles
(1883,) n_aromatic_rings
(1883,) n_saturated_carbocycles
(1883,) n_saturated_heterocyles
(1883,) n_saturated_rings


In [35]:
res_svm_cross = []
for i in range(y_train.shape[1]):
    y_svm = y_train.iloc[:, i]
    svm_cross = -cross_val_score(svm_reg, X_train, y_svm, cv=5, n_jobs=-1,scoring='neg_root_mean_squared_error')
    res_svm_cross.append(svm_cross.mean())


In [36]:
res_svm_cross

[np.float64(0.7934635648923551),
 np.float64(0.06438874327232462),
 np.float64(0.5045715443367023),
 np.float64(0.22906066252215035),
 np.float64(0.3725972898725145),
 np.float64(0.5890841628037069),
 np.float64(2.597579276696109),
 np.float64(0.7808628040182175),
 np.float64(13.543161642880563),
 np.float64(0.07813145917289488),
 np.float64(0.6038289372157575),
 np.float64(0.18616188341298645),
 np.float64(0.2479759162874707),
 np.float64(0.20356142840450073),
 np.float64(0.2810086240494201),
 np.float64(0.3759420972925439),
 np.float64(0.3461131160525712),
 np.float64(0.36434226167816663),
 np.float64(0.2292262976665871),
 np.float64(0.15452926349151624),
 np.float64(0.2296929512658054)]

In [37]:
svm_cross_df = pd.Series(res_svm_cross, index=col_y, name='svm')
svm_cross_df

pKi                         0.793464
fsp3                        0.064389
n_lipinski_hba              0.504572
n_lipinski_hbd              0.229061
n_rings                     0.372597
n_hetero_atoms              0.589084
n_heavy_atoms               2.597579
n_rotatable_bonds           0.780863
tpsa                       13.543162
qed                         0.078131
clogp                       0.603829
sas                         0.186162
n_aliphatic_carbocycles     0.247976
n_aliphatic_heterocyles     0.203561
n_aliphatic_rings           0.281009
n_aromatic_carbocycles      0.375942
n_aromatic_heterocyles      0.346113
n_aromatic_rings            0.364342
n_saturated_carbocycles     0.229226
n_saturated_heterocyles     0.154529
n_saturated_rings           0.229693
Name: svm, dtype: float64

In [38]:
res_df = pd.merge(rf_cros_df, svm_cross_df, left_index=True, right_index=True)
res_df.reset_index(inplace=True, names='descriptors')
res_df

,descriptors,random_forest,svm
0,pKi,0.739326,0.793464
1,fsp3,0.044040,0.064389
2,n_lipinski_hba,0.552972,0.504572
3,n_lipinski_hbd,0.200272,0.229061
4,n_rings,0.373395,0.372597
5,n_hetero_atoms,0.659574,0.589084
6,n_heavy_atoms,2.215775,2.597579
7,n_rotatable_bonds,0.728725,0.780863
8,tpsa,7.191903,13.543162
9,qed,0.067386,0.078131


In [39]:
cwd = Path.cwd()
work_dir = cwd / 'regressors'
work_dir.mkdir(exist_ok=True)
files = []
for file in work_dir.iterdir():
    files.append(file)
for file in files:
    file.unlink()

In [40]:
regressors_pieplines = dict()

In [41]:
for i in range(y_train.shape[1]):
    name = y_train.iloc[:, i].name
    tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2, random_state=42, n_jobs=-1)
    tpot.fit(X_train, y_train.iloc[:, i])
    print(f'{name}: {tpot.score(X_val, y_val.iloc[:, i])}')
    regressors_pieplines[name] = tpot
    filename = work_dir / f'{name}_regressor.py'
    tpot.export(filename)
    

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.5340500650185827

Generation 2 - Current best internal CV score: -0.5340500650185827

Generation 3 - Current best internal CV score: -0.5340500650185827

Generation 4 - Current best internal CV score: -0.5340500650185827

Generation 5 - Current best internal CV score: -0.5340500650185827

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.2, min_samples_leaf=1, min_samples_split=7, n_estimators=100)
pKi: -0.49656080147884496


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.0016616163633071171

Generation 2 - Current best internal CV score: -0.0016615068549424173

Generation 3 - Current best internal CV score: -0.0015920534091922716

Generation 4 - Current best internal CV score: -0.0014380093427504962

Generation 5 - Current best internal CV score: -0.0014380093427504962

Best pipeline: ExtraTreesRegressor(MaxAbsScaler(RidgeCV(input_matrix)), bootstrap=False, max_features=0.2, min_samples_leaf=1, min_samples_split=7, n_estimators=100)
fsp3: -0.0014141587660542354


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]

...


Generation 1 - Current best internal CV score: -0.025250866394192578

Generation 2 - Current best internal CV score: -0.022064948578133542

Generation 3 - Current best internal CV score: -0.022064948578133542

Generation 4 - Current best internal CV score: -0.019346262779170044

Generation 5 - Current best internal CV score: -0.019346262779170044

Best pipeline: ExtraTreesRegressor(RidgeCV(input_matrix), bootstrap=False, max_features=0.6000000000000001, min_samples_leaf=3, min_samples_split=7, n_estimators=100)
n_saturated_carbocycles: -0.030956705151328055


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.016936590844220892

Generation 2 - Current best internal CV score: -0.016936590844220892

Generation 3 - Current best internal CV score: -0.016936590844220892

Generation 4 - Current best internal CV score: -0.016936590844220892

Generation 5 - Current best internal CV score: -0.01672431253354842

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.4, min_samples_leaf=1, min_samples_split=7, n_estimators=100)
n_saturated_heterocyles: -0.016843350908233072


Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.031844067936803555

Generation 2 - Current best internal CV score: -0.031844067936803555

Generation 3 - Current best internal CV score: -0.031844067936803555

Generation 4 - Current best internal CV score: -0.031844067936803555

Generation 5 - Current best internal CV score: -0.031844067936803555

Best pipeline: ExtraTreesRegressor(input_matrix, bootstrap=False, max_features=0.2, min_samples_leaf=1, min_samples_split=7, n_estimators=100)
n_saturated_rings: -0.028668958008964386


In [42]:
regressors_pieplines['n_saturated_rings'].fitted_pipeline_.get_params()

{'memory': None,
 'steps': [('extratreesregressor',
   ExtraTreesRegressor(max_features=0.2, min_samples_split=7, random_state=42))],
 'verbose': False,
 'extratreesregressor': ExtraTreesRegressor(max_features=0.2, min_samples_split=7, random_state=42),
 'extratreesregressor__bootstrap': False,
 'extratreesregressor__ccp_alpha': 0.0,
 'extratreesregressor__criterion': 'squared_error',
 'extratreesregressor__max_depth': None,
 'extratreesregressor__max_features': 0.2,
 'extratreesregressor__max_leaf_nodes': None,
 'extratreesregressor__max_samples': None,
 'extratreesregressor__min_impurity_decrease': 0.0,
 'extratreesregressor__min_samples_leaf': 1,
 'extratreesregressor__min_samples_split': 7,
 'extratreesregressor__min_weight_fraction_leaf': 0.0,
 'extratreesregressor__monotonic_cst': None,
 'extratreesregressor__n_estimators': 100,
 'extratreesregressor__n_jobs': None,
 'extratreesregressor__oob_score': False,
 'extratreesregressor__random_state': 42,
 'extratreesregressor__verbose'

In [43]:
pipelines = dict()
for key, value in regressors_pieplines.items():
    pipelines[key] = value.fitted_pipeline_

In [44]:
pipelines

{'pKi': Pipeline(steps=[('extratreesregressor',
                  ExtraTreesRegressor(max_features=0.2, min_samples_split=7,
                                      random_state=42))]),
 'fsp3': Pipeline(steps=[('stackingestimator', StackingEstimator(estimator=RidgeCV())),
                 ('maxabsscaler', MaxAbsScaler()),
                 ('extratreesregressor',
                  ExtraTreesRegressor(max_features=0.2, min_samples_split=7,
                                      random_state=42))]),
 'n_lipinski_hba': Pipeline(steps=[('xgbregressor',
                  XGBRegressor(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
          

In [45]:
pipelines['tpsa'].get_params()

{'memory': None,
 'steps': [('polynomialfeatures', PolynomialFeatures(include_bias=False)),
  ('ridgecv', RidgeCV())],
 'verbose': False,
 'polynomialfeatures': PolynomialFeatures(include_bias=False),
 'ridgecv': RidgeCV(),
 'polynomialfeatures__degree': 2,
 'polynomialfeatures__include_bias': False,
 'polynomialfeatures__interaction_only': False,
 'polynomialfeatures__order': 'C',
 'ridgecv__alpha_per_target': False,
 'ridgecv__alphas': (0.1, 1.0, 10.0),
 'ridgecv__cv': None,
 'ridgecv__fit_intercept': True,
 'ridgecv__gcv_mode': None,
 'ridgecv__scoring': None,
 'ridgecv__store_cv_results': None,
 'ridgecv__store_cv_values': 'deprecated'}

In [46]:
pickle_dir = cwd / 'pickles'
pickle_dir.mkdir(exist_ok=True)

In [47]:
for name, regressor in pipelines.items():
    filename = pickle_dir / f'{name}.pickle'
    with open(filename, 'wb') as file:
        pickle.dump(regressor, file, pickle.HIGHEST_PROTOCOL)

In [48]:
data.to_csv('data_descriptors.csv')

In [49]:
res_df.to_csv('regressors_comparision.csv')